In [ ]:
def format_input(entry):
    """
    Alpaca 스타일의 프롬프트 템플릿을 적용하는 함수.
    모델에게 역할을 부여하고 입력 형식을 통일합니다.
    """
    instruction_text = (
        f"Below is an instruction that describes a task. "
        f"Write a response that appropriately completes the request."
        f"\n\n### Instruction:\n{entry['instruction']}"
    )

    # 추가적인 입력 정보(Context 등)가 있는 경우 추가
    input_text = f"\n\n### Input:\n{entry['input']}" if entry["input"] else ""

    return instruction_text + input_text


In [ ]:
class InstructionDataset(Dataset):
    """
    지시사항(Instruction) 데이터셋을 처리하는 PyTorch Dataset 클래스.
    입력 데이터(지시+입력)와 정답 데이터(응답)를 하나의 텍스트로 합쳐서 토큰화합니다.
    """
    def __init__(self, data, tokenizer):
        self.data = data

        # 텍스트 데이터 미리 토큰화 (Pre-tokenize)
        self.encoded_texts = []
        for entry in data:
            # 포맷팅 함수를 이용해 "지시사항 + 입력" 텍스트 생성
            instruction_plus_input = format_input(entry)
            # 정답(Response) 텍스트 생성
            response_text = f"\n\n### Response:\n{entry['output']}"
            
            # 모델은 이 전체 텍스트(질문+답변)를 보고 다음 토큰을 예측하도록 학습됨
            full_text = instruction_plus_input + response_text
            self.encoded_texts.append(
                tokenizer.encode(full_text)
            )

In [ ]:
def custom_collate_fn( batch,
    pad_token_id=50256, ignore_index=-100, allowed_max_length=None,
    device="cpu"
):
    """
    데이터 로더에서 배치를 만들 때 사용하는 커스텀 함수.
    가변 길이의 시퀀스를 배치의 최대 길이에 맞춰 패딩(padding)하고, 
    정답(target) 데이터에서 패딩 부분은 손실(loss) 계산에서 제외하도록 처리합니다.
    """
    # 배치 내에서 가장 긴 시퀀스 길이 계산 (패딩을 위해 +1 여유)
    batch_max_length = max(len(item)+1 for item in batch)
    
    # 입력(inputs)과 정답(targets) 리스트 준비
    inputs_lst, targets_lst = [], []
    
    for item in batch:
        new_item = item.copy()
        # 문장 끝 토큰 <|endoftext|> 추가
        new_item += [pad_token_id]
        
        # 가장 긴 길이에 맞춰 패딩 추가
        padded = new_item + [pad_token_id] * (batch_max_length - len(new_item))
        
        # 입력은 마지막 토큰 제외 (0 ~ n-1)
        inputs = torch.tensor(padded[:-1]) 
        # 정답은 첫 번째 토큰 제외 (1 ~ n) -> 다음 토큰 예측 문제이므로
        targets = torch.tensor(padded[1:]) 

        # 중요: 패딩 부분 마스킹 처리
        # 타겟에서 패딩 토큰인 부분은 ignore_index(-100)로 바꿔서 CrossEntropyLoss 계산 시 무시되게 함
        mask = targets == pad_token_id
        indices = torch.nonzero(mask).squeeze()
        
        # 패딩이 시작되는 지점 이후의 모든 타겟 값을 -100으로 설정
        if indices.numel() > 1:
            targets[indices[1:]] = ignore_index
        # indices[1:] → 첫 번째 pad 위치를 제외한 나머지 pad 위치 인덱스 
        # 위치의 targets 값을 ignore_index로 바꿉니다.
        
        # 선택적으로 최대 시퀀스 길이 제한 (메모리 관리 등 목적)
        if allowed_max_length is not None:
            inputs = inputs[:allowed_max_length]
            targets = targets[:allowed_max_length]

        inputs_lst.append(inputs)
        targets_lst.append(targets)
    
    # 텐서로 변환 및 디바이스(GPU/CPU)로 이동
    inputs_tensor = torch.stack(inputs_lst).to(device)
    targets_tensor = torch.stack(targets_lst).to(device)

    return inputs_tensor, targets_tensor
    